In [341]:
import io
import math
import re

import numpy as np
np.set_printoptions(precision=2)

from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
pd.options.display.max_rows = 6
pd.set_option('display.precision', 2)

from IPython.display import display, HTML, Markdown
def markdown(md): display(Markdown(md))

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_white'

# Loading the dataset

In [9]:
try:
    pd.read_csv('data/vast challenge 2011/Microblogs.csv')
except FileNotFoundError:
    VASTOPOLIS_URL = 'https://drive.google.com/file/d/1Y5xWPRKk8D_TNYmZ0ny4cQ83shGM5YqI/view?usp=sharing'
    VASTOPOLIS_URL = 'https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(VASTOPOLIS_URL.split('/')[-2])

    Posts = pd.read_csv(VASTOPOLIS_URL, encoding='latin', on_bad_lines='skip')

# Convert, sort, and index the messages by the date/time of creation
Posts.Created_at = pd.to_datetime(Posts.Created_at, format='%m/%d/%Y %H:%M', errors='coerce')
Posts.dropna(inplace=True)  # Drop items that have N/A values
Posts.sort_values(by=['Created_at', 'ID'], inplace=True)
Posts.index = Posts.Created_at

# Extract the latitude and longitude from the Location column
Posts[['Latitude', 'Longitude']] = Posts.Location.str.split(' ', n=1, expand=True).astype('float')
Posts.drop('Location', axis='columns', inplace=True)
Posts.Longitude = -Posts.Longitude

# Create a grid of cells and assign each message to a cell
Bounds = dict(east=Posts.Longitude.max(), west=Posts.Longitude.min(), north=Posts.Latitude.max(), south=Posts.Latitude.min())

N_C = 12
N_R = int(N_C * (Bounds['north'] - Bounds['south']) / (Bounds['east'] - Bounds['west']))
Posts['R'] = ((Posts.Latitude - Bounds['south']) / (Bounds['north'] - Bounds['south']) * N_R).astype('int').astype('str')
Posts['C'] = ((Posts.Longitude - Bounds['west']) / (Bounds['east'] - Bounds['west']) * N_C).astype('int').map(lambda x: chr(ord('A') + x))

# Symptoms

Symptoms = 'fever chill sweat ache fatigue breath pain cough vomit nausea'.split(' ')

for symptom in Symptoms:
    Posts[symptom] = Posts.text.str.contains(symptom)
Posts['symptoms'] = Posts[Symptoms].any(axis='columns')
Posts['symptom'] = ''
for symptom in Symptoms:
    Posts.loc[Posts[symptom], 'symptom'] = symptom

# Query posts

In [10]:
def query_posts(posts=Posts, start_date=None, end_date=None, area=None, symptoms=None, keywords=None, people=None):
    if start_date is not None:
        posts = posts.loc[start_date:end_date] if end_date is not None else posts.loc[start_date]

    if area is not None:
        c1, r1, c2, r2 = re.match(r'([A-M])(\d+):([A-M])(\d+)', area.upper()).groups()
        q_area = f'("{r1}" <= R < "{r2}") and ("{c1}" <= C < "{c2}")'
    else: q_area = None

    if symptoms is not None:
        q_symptoms = f'({" | ".join(symptoms)})'
    else: q_symptoms = None

    q = list(filter(None, [q_area, q_symptoms]))
    if len(q) > 0:
        # print(q)
        posts = posts.query(' & '.join(q))

    if type(keywords) == list:
        pattern = re.compile('|'.join(map(re.escape, keywords)))
        # print(pattern)
        posts = posts[posts.text.str.contains(pattern)]
    
    if people is not None:    posts = posts[posts.ID.isin(people)]

    return posts.copy()

In [90]:
def label_posts_with_keywords(posts, label, keywords):
    posts[label] = ''
    for kwd in keywords:
        posts.loc[posts.text.str.contains(kwd), label] = kwd
    return posts

In [166]:
def show_trend(posts):
    trend = posts.Created_at.dt.hour.value_counts().sort_index()
    display(px.bar(trend, x=trend.index, y=trend.values, labels={'x': 'Hour', 'y': '#Posts'}))

In [192]:
def show_post_texts(posts, N=15):
    if len(posts) > N:
        posts = posts.sample(N).sort_index()
    md = '\n\n'.join([f'- {t}: {text.text}' for t, text in posts.iterrows()])
    markdown(md)

# Constants

In [11]:
Day1 = '2011-05-18'
Day2 = '2011-05-19'
Day3 = '2011-05-20'

OutbreakArea1 = 'F1:I3'
OutbreakArea2 = 'A0:F2'

# Map and animation

In [136]:
WIDTH = 1600  # Adjust the width of the map to fit your display size
HEIGHT = int(WIDTH * (Bounds['north'] - Bounds['south']) / (Bounds['east'] - Bounds['west']))
IMAGE_URL = 'https://wakita.github.io/smartnova/assets/images/Vastopolis_Map.png'

MapTicktextX = [chr(ord('A') + n) for n in range(N_C + 1)]
MapTicktextY = list(range(N_R+1))
MapTickvalsX = np.linspace(Bounds['west'], Bounds['east'], N_C + 1).tolist()
MapTickvalsY = np.linspace(Bounds['south'], Bounds['north'], N_R + 1).tolist()

def create_map(messages, **extraargs):
    fig = px.scatter(messages, x='Longitude', y='Latitude', width=WIDTH, height=HEIGHT, hover_name='ID', hover_data=['Created_at', 'R', 'C', 'text'], **extraargs)
    fig.update_traces(marker={'size': 4})

    fig.add_layout_image(
            dict(
                source=IMAGE_URL,
                xref='x', yref='y', x=Bounds['west'], y=Bounds['north'],
                sizex=Bounds['east'] - Bounds['west'], sizey=Bounds['north'] - Bounds['south'],  sizing='stretch',
                layer='below', opacity=0.5))

    fig.update_xaxes(range=[Bounds['west'], Bounds['east']],
                     tickmode='array', ticktext=MapTicktextX, tickvals=MapTickvalsX)
    fig.update_yaxes(scaleanchor='x', scaleratio = 1,
                     range=[Bounds['south'], Bounds['north']],
                     tickmode='array', ticktext=MapTicktextY, tickvals=MapTickvalsY)

    return fig

def animate_map(posts, by='day', **extraargs):
    if by == 'day':
        posts[by] = posts.Created_at.dt.strftime('%Y-%m-%d')
    elif by == 'hour':
        origin = posts.iloc[0].Created_at
        print(origin)
        posts[by] = (posts.Created_at - origin).dt.days * 24 + (posts.Created_at - origin).dt.seconds // 3600

    fig = create_map(posts, animation_frame=by, **extraargs)
    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 2000
    fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 0

    fig.show()

# TF-IDF

In [132]:
Vectorizer = TfidfVectorizer(min_df=10, max_df=0.5, stop_words='english')
Vectorizer.fit(Posts.text.values)
Words = Vectorizer.get_feature_names_out()    # The words in the vocabulary

def tfidf(documents):
    return Vectorizer.transform(documents)

CorpusNormalDays = ' '.join(Posts.loc[:'2011-05-15'].text.values)
normal_tfidf = tfidf([CorpusNormalDays])

In [352]:
def summarize_posts(title, posts):
    tfidf_values = tfidf([' '.join(posts.text.values)])
    
    notable_words = [(tfidf_values[0, i], Words[i]) for i in tfidf_values.nonzero()[1]
                     if tfidf_values[0, i] - normal_tfidf[0, i] > 0.01]
    notable_words = sorted(notable_words, reverse=True)

    html = '<blockquote>' + ' '.join([f'<span style="font-size: {max(int(math.log(v * 200) * 6), 10)}px">{word}</span>' for v, word in notable_words]) + '</blockquote>'
    markdown(f'### {title} ({len(posts)} messages)\n\n' + html)
    #markdown(f'### {title} ({len(posts)} messages)')
    #display(HTML(html))

def topics(posts, title=None, format='txt'):
    tfidf_values = tfidf([' '.join(posts.text.values)])
    notable_words = [(tfidf_values[0, i], Words[i]) for i in tfidf_values.nonzero()[1]
                    if tfidf_values[0, i] - normal_tfidf[0, i] > 0.01]
    notable_words = sorted(notable_words, reverse=True)

    if format == 'html':
        title = f'### {title} ({len(posts)} messages)\n\n' if title else ''
        html = '<blockquote>' + ' '.join([f'<span style="font-size: {max(int(math.log(v * 200) * 6), 10)}px">{word}</span>' for v, word in notable_words]) + '</blockquote>'
        return Markdown(title + html)
    
    summary = ' '.join([word for v, word in notable_words])
    return summary if len(summary) < 200 else summary[:200]


In [356]:
display(topics(Posts.loc['2011-05-16'], title='2011-05-16', format='html'))
display(topics(Posts.loc['2011-05-17'], title='2011-05-17', format='html'))

### 2011-05-16 (48966 messages)

<blockquote><span style="font-size: 22px">comic</span> <span style="font-size: 22px">convention</span> <span style="font-size: 21px">bomb</span> <span style="font-size: 20px">day</span> <span style="font-size: 20px">today</span> <span style="font-size: 18px">heard</span> <span style="font-size: 18px">news</span> <span style="font-size: 18px">said</span> <span style="font-size: 17px">city</span> <span style="font-size: 16px">bombs</span> <span style="font-size: 16px">threats</span> <span style="font-size: 16px">told</span> <span style="font-size: 16px">better</span> <span style="font-size: 16px">informed</span> <span style="font-size: 15px">hope</span> <span style="font-size: 15px">pretty</span> <span style="font-size: 15px">filled</span> <span style="font-size: 14px">away</span> <span style="font-size: 14px">baseball</span> <span style="font-size: 14px">end</span> <span style="font-size: 13px">nice</span> <span style="font-size: 13px">stay</span> <span style="font-size: 13px">awesome</span> <span style="font-size: 13px">stuff</span> <span style="font-size: 13px">crews</span> <span style="font-size: 13px">cool</span> <span style="font-size: 12px">recently</span> <span style="font-size: 12px">soon</span> <span style="font-size: 12px">convetion</span> <span style="font-size: 11px">needs</span> <span style="font-size: 11px">taking</span> <span style="font-size: 10px">happen</span> <span style="font-size: 10px">idea</span> <span style="font-size: 10px">possible</span> <span style="font-size: 10px">bought</span> <span style="font-size: 10px">started</span> <span style="font-size: 10px">police</span> <span style="font-size: 10px">hurt</span> <span style="font-size: 10px">threat</span> <span style="font-size: 10px">longer</span> <span style="font-size: 10px">rampant</span> <span style="font-size: 10px">huge</span> <span style="font-size: 10px">gearing</span> <span style="font-size: 10px">deployed</span> <span style="font-size: 10px">boms</span> <span style="font-size: 10px">arising</span> <span style="font-size: 10px">preping</span> <span style="font-size: 10px">cover</span> <span style="font-size: 10px">searching</span> <span style="font-size: 10px">blow</span> <span style="font-size: 10px">buildings</span> <span style="font-size: 10px">hopefully</span> <span style="font-size: 10px">emergency</span> <span style="font-size: 10px">explosion</span> <span style="font-size: 10px">scare</span> <span style="font-size: 10px">toes</span> <span style="font-size: 10px">drill</span> <span style="font-size: 10px">peaceful</span> <span style="font-size: 10px">scary</span> <span style="font-size: 10px">places</span> <span style="font-size: 10px">area</span> <span style="font-size: 10px">possibly</span> <span style="font-size: 10px">dangerous</span> <span style="font-size: 10px">hero</span> <span style="font-size: 10px">reported</span> <span style="font-size: 10px">large</span> <span style="font-size: 10px">cautious</span> <span style="font-size: 10px">expected</span> <span style="font-size: 10px">suggest</span> <span style="font-size: 10px">rumors</span> <span style="font-size: 10px">chaotic</span> <span style="font-size: 10px">tweeted</span> <span style="font-size: 10px">everyinthing</span> <span style="font-size: 10px">crowded</span> <span style="font-size: 10px">attendance</span> <span style="font-size: 10px">hook</span> <span style="font-size: 10px">danger</span></blockquote>

### 2011-05-17 (46524 messages)

<blockquote><span style="font-size: 18px">friend</span> <span style="font-size: 18px">smog</span> <span style="font-size: 18px">town</span> <span style="font-size: 15px">truck</span> <span style="font-size: 15px">car</span> <span style="font-size: 15px">felt</span> <span style="font-size: 15px">big</span> <span style="font-size: 15px">heard</span> <span style="font-size: 15px">saw</span> <span style="font-size: 14px">explosion</span> <span style="font-size: 13px">believe</span> <span style="font-size: 13px">earth</span> <span style="font-size: 13px">smoke</span> <span style="font-size: 13px">accident</span> <span style="font-size: 13px">place</span> <span style="font-size: 13px">trucks</span> <span style="font-size: 13px">610</span> <span style="font-size: 12px">explosions</span> <span style="font-size: 12px">boom</span> <span style="font-size: 11px">shake</span> <span style="font-size: 11px">noon</span> <span style="font-size: 11px">traffic</span> <span style="font-size: 11px">safe</span> <span style="font-size: 10px">scary</span> <span style="font-size: 10px">wind</span> <span style="font-size: 10px">definetly</span> <span style="font-size: 10px">ground</span> <span style="font-size: 10px">hurt</span> <span style="font-size: 10px">terrible</span> <span style="font-size: 10px">huge</span> <span style="font-size: 10px">rattle</span> <span style="font-size: 10px">shook</span> <span style="font-size: 10px">270</span> <span style="font-size: 10px">cloud</span> <span style="font-size: 10px">anymore</span> <span style="font-size: 10px">blast</span> <span style="font-size: 10px">wrecked</span> <span style="font-size: 10px">destruction</span> <span style="font-size: 10px">bridge</span> <span style="font-size: 10px">careful</span> <span style="font-size: 10px">massive</span> <span style="font-size: 10px">dust</span> <span style="font-size: 10px">tons</span> <span style="font-size: 10px">unsafe</span> <span style="font-size: 10px">smashed</span> <span style="font-size: 10px">breathing</span></blockquote>

# Analysis for 2011-05-17

In [95]:
def _():
    keywords = 'friend smog town truck car explosion earth smoke accident 610'.split(' ')
    posts = query_posts(posts=Posts.loc['2011-05-17'], keywords=keywords)
    label_posts_with_keywords(posts, 'keyword', keywords)
    display(create_map(query_posts(posts=posts), color='keyword'))

_()

## Three areas of interest

From the clustering of points on the map, we find three interesting areas. For each area, we listed characteristic keywords by toggling the display of keywords by operating the legend.

1. Smog-town (explosion, smoke, earth, town, friend, car)

2. A bridge on road 270 (accident, car)

3. A bridge on road 610 (610, accident, truck, car)

We see higher number of posts in the city center but the rate of congestion is not high. Probably they are normal posts.

In [359]:
def show_trend(posts):
    date = posts.iloc[0].Created_at.strftime('%Y-%m-%d')

    def topics(posts):
        tfidf_values = tfidf([' '.join(posts.text.values)])
        notable_words = [(tfidf_values[0, i], Words[i]) for i in tfidf_values.nonzero()[1]
                        if tfidf_values[0, i] - normal_tfidf[0, i] > 0.01]
        notable_words = sorted(notable_words, reverse=True)
        summary = ' '.join([word for v, word in notable_words])
        return summary if len(summary) < 200 else summary[:200]

    trend = posts.Created_at.dt.hour.value_counts().sort_index()
    trend = pd.DataFrame(dict(hour=trend.index, count=trend.values, topic=[topics(posts.loc[f'{date} {t-1}:00':f'{date} {t}:00']) for t in trend.index]))
    fig = px.bar(trend, x='hour', y='count', labels={'x': 'Hour', 'y': '#Posts'}, custom_data=['topic'])
    fig.update_traces(hovertemplate='%{customdata[0]}')
    display(fig)

def event_report(title, date, area, keywords):
    posts = query_posts(posts=Posts.loc[date], area=area, keywords=keywords)
    posts = label_posts_with_keywords(posts, 'keyword', keywords)
    display(topics(posts, title=title, format='html'))
    show_trend(posts)
    display(create_map(posts, color='keyword'))

def hourly_report(posts, date, start, end):
    for t in range(start, end):
        display(topics(posts.loc[f'2011-05-17 {t}:00':f'2011-05-17 {t+1}:00'], title=f'2011-05-17 {t}:00', format='html'))
        #summarize_posts(f'2011-05-17 {t}:00', )

---
## Explosion on Smogtown (explosion, smoke, earth, town, friend, car)

In [360]:
event_report('Smogtown', '2011-05-17', 'a0:b1', 'friend explosion earth boom shake rattle blast smoke'.split(' '))

### Smogtown (1087 messages)

<blockquote><span style="font-size: 27px">smog</span> <span style="font-size: 26px">town</span> <span style="font-size: 25px">friend</span> <span style="font-size: 23px">felt</span> <span style="font-size: 23px">explosion</span> <span style="font-size: 23px">heard</span> <span style="font-size: 21px">explosions</span> <span style="font-size: 21px">saw</span> <span style="font-size: 21px">earth</span> <span style="font-size: 20px">boom</span> <span style="font-size: 20px">shake</span> <span style="font-size: 19px">definetly</span> <span style="font-size: 18px">rattle</span> <span style="font-size: 18px">shook</span> <span style="font-size: 18px">safe</span> <span style="font-size: 18px">big</span> <span style="font-size: 17px">scary</span> <span style="font-size: 17px">ground</span> <span style="font-size: 17px">blast</span> <span style="font-size: 17px">cloud</span> <span style="font-size: 16px">wind</span> <span style="font-size: 16px">place</span> <span style="font-size: 16px">smoke</span> <span style="font-size: 15px">hope</span> <span style="font-size: 15px">huge</span> <span style="font-size: 15px">unsafe</span> <span style="font-size: 15px">careful</span> <span style="font-size: 14px">massive</span> <span style="font-size: 14px">tons</span> <span style="font-size: 14px">dust</span> <span style="font-size: 14px">alright</span> <span style="font-size: 13px">anymore</span> <span style="font-size: 12px">hurt</span> <span style="font-size: 12px">city</span> <span style="font-size: 12px">breathing</span></blockquote>

In [361]:
show_post_texts(query_posts(posts=PostsOnSmogtown.loc['2011-05-17 09:00':'2011-05-17 09:30']))

- 2011-05-17 09:00:00: My friend saw that there is a huge smoke cloud in Smog Town smog town is definetly not the place to be

- 2011-05-17 09:00:00: so much for a safe city... I saw an explosion

- 2011-05-17 09:01:00: I heard  the ground shake explosions are scary

- 2011-05-17 09:04:00: My friend saw some big boom Boom!

- 2011-05-17 09:07:00: Smog Town is definetly unsafe My friend saw Smog town had an explosion

- 2011-05-17 09:08:00: I hope everyone is alright I saw Smog town had an explosion

- 2011-05-17 09:10:00: is anywhere safe anymore? My friend felt  the ground shake

- 2011-05-17 09:11:00: I hope everyone is alright I felt that there was a huge explosion

- 2011-05-17 09:13:00: My friend saw that there is a huge smoke cloud in Smog Town I hope everyone is alright

- 2011-05-17 09:17:00: I heard from a friend the earth shook in Smog Town massive smoke cloud

- 2011-05-17 09:18:00: explosions are scary My friend felt an explosion

- 2011-05-17 09:22:00: I felt that the ground was shaking in Smog Town massive smoke cloud

- 2011-05-17 09:25:00: explosions are scary I saw a blast of wind

- 2011-05-17 09:29:00: smog town is definetly not the place to be My friend felt an explosion

- 2011-05-17 09:29:00: My friend heard Smog town had an explosion hope noone got hurt

### A tentative summary on the explosion in Smogtown

- People are reporting explosions (blast, rattle, boom, smoke, earth, etc.) in the center of Smogtown and many care about friends.

- The cause of the explosion needs to be identified.

- We need to investigate this event in connection with the spread of the health problem.

---
## A bridge on route 270 (accident, car)

In [362]:
event_report('Car accident on route 270', '2011-05-17', 'c1:e2', 'car accident'.split(' '))

### Car accident on route 270 (311 messages)

<blockquote><span style="font-size: 29px">car</span> <span style="font-size: 27px">accident</span> <span style="font-size: 25px">270</span> <span style="font-size: 21px">traffic</span> <span style="font-size: 18px">terrible</span> <span style="font-size: 18px">destruction</span> <span style="font-size: 18px">believe</span> <span style="font-size: 17px">wrecked</span> <span style="font-size: 17px">bad</span> <span style="font-size: 16px">bridge</span> <span style="font-size: 15px">swear</span> <span style="font-size: 15px">gah</span> <span style="font-size: 15px">escape</span> <span style="font-size: 14px">stupid</span> <span style="font-size: 14px">stuck</span> <span style="font-size: 13px">anytime</span> <span style="font-size: 13px">smashed</span> <span style="font-size: 12px">saw</span> <span style="font-size: 12px">caused</span> <span style="font-size: 12px">destroyed</span> <span style="font-size: 12px">totally</span> <span style="font-size: 12px">soon</span> <span style="font-size: 11px">trashed</span> <span style="font-size: 11px">irrepairable</span> <span style="font-size: 10px">burning</span> <span style="font-size: 10px">feb</span> <span style="font-size: 10px">witnessed</span></blockquote>

In [363]:
show_post_texts(query_posts(posts=PostsOnRoad270.loc['2011-05-17 08:00':'2011-05-17 09:30']))

- 2011-05-17 08:47:00: Fire up the engines...#Nascar

- 2011-05-17 09:10:00: I can't believe the terrible car accident I just saw such terrible destruction

- 2011-05-17 09:12:00: That car is destroyed this stupid bridge 270 I swear

- 2011-05-17 09:14:00: such terrible destruction That car has been on fire.

- 2011-05-17 09:15:00: That car has been on fire. is there any escape from 270!?

- 2011-05-17 09:18:00: That car is destroyed such terrible destruction

- 2011-05-17 09:21:00: Flew out da windshield in a car accident.  Had Surgery 32 times just on my throat.  And Been in a coma. IM NOT LUCKY  IM BLESS   #toptha

- 2011-05-17 09:21:00: Got kicked out of mu house for getting in a car accident

- 2011-05-17 09:23:00: this stupid bridge 270 I swear That car has been on fire.

- 2011-05-17 09:25:00: My good friend  Marc  was killed in a car accident a few hours ago. I am taking a break from Twitter for a few days. &lt;3 u all

- 2011-05-17 09:26:00: Just saw a car accident! Dang

- 2011-05-17 09:27:00: If imm lucky ill get into a car accident so i dont have to listen to you -__- #ihateyou

- 2011-05-17 09:28:00: There are a lot of car accident related deaths here...

- 2011-05-17 09:29:00: Horrible car accident 41 and 5... I everyone is alright

- 2011-05-17 09:30:00: Just saw a car accident right in front of me on the freeway...that was a little freaky

### A summary on the accident near the bridge on route 270

- A regular car accident near a bridge on route/freeway 270 at about 09:10.  (08:47 message is probably unrelated to the accident but you may want to check if it was posted by the poor driver.)

- This event is unrelated to the spread of the health problem.

---
## Truck accident near a bridge on road 610 (610, accident, truck, car)



In [364]:
event_report('Truck accident near a bridge on road 610', '2011-05-17', 'e1:f2', '610 accident truck car'.split(' '))

### Truck accident near a bridge on road 610 (381 messages)

<blockquote><span style="font-size: 29px">truck</span> <span style="font-size: 26px">610</span> <span style="font-size: 25px">trucks</span> <span style="font-size: 24px">noon</span> <span style="font-size: 21px">accident</span> <span style="font-size: 20px">believe</span> <span style="font-size: 19px">traffic</span> <span style="font-size: 18px">wrecked</span> <span style="font-size: 18px">terrible</span> <span style="font-size: 18px">fun</span> <span style="font-size: 17px">destruction</span> <span style="font-size: 16px">hated</span> <span style="font-size: 15px">lunch</span> <span style="font-size: 15px">bridge</span> <span style="font-size: 15px">totally</span> <span style="font-size: 15px">spilling</span> <span style="font-size: 15px">burning</span> <span style="font-size: 15px">cargo</span> <span style="font-size: 15px">tipped</span> <span style="font-size: 15px">timing</span> <span style="font-size: 15px">especially</span> <span style="font-size: 14px">12pm</span> <span style="font-size: 14px">drivers</span> <span style="font-size: 14px">anytime</span> <span style="font-size: 14px">smashed</span> <span style="font-size: 13px">trashed</span> <span style="font-size: 13px">caused</span> <span style="font-size: 13px">perfect</span> <span style="font-size: 13px">road</span> <span style="font-size: 13px">irrepairable</span> <span style="font-size: 13px">drive</span> <span style="font-size: 13px">esacape</span> <span style="font-size: 13px">dangerous</span> <span style="font-size: 13px">soon</span> <span style="font-size: 13px">saw</span> <span style="font-size: 12px">destroyed</span> <span style="font-size: 12px">sitting</span> <span style="font-size: 12px">isn</span> <span style="font-size: 12px">camping</span> <span style="font-size: 11px">backed</span> <span style="font-size: 11px">agree</span> <span style="font-size: 11px">happen</span> <span style="font-size: 10px">moving</span> <span style="font-size: 10px">starving</span></blockquote>

In [365]:
show_post_texts(PostsOnRoad610.loc['2011-05-17 09:00':'2011-05-17 11:20'])

- 2011-05-17 10:41:00: All I care about is how Punk Rock I feel when I wake up in the morning.

- 2011-05-17 11:00:00: I always hated trucks especially at noon now That truck has been irrepairable

- 2011-05-17 11:02:00: That truck has been spilling its cargo traffic will never move now on 610

- 2011-05-17 11:05:00: traffic will never move now on 610 Trucks are not gonna move again anytime soon

- 2011-05-17 11:08:00: trucks seem so dangerous more so at noon to me now I can't believe the terrible truck accident I just saw

- 2011-05-17 11:10:00: Trucks are smashed up 12pm and sitting on 610 fun fun

- 2011-05-17 11:10:00: Trucks are burning. I always hated trucks especially at noon now

- 2011-05-17 11:13:00: oh why did this have to happen at noon That truck is totally wrecked

- 2011-05-17 11:15:00: such terrible destruction and just in time for lunch That truck has been on fire.

- 2011-05-17 11:16:00: Trucks are spilling its cargo 610 isn't looking to fun right about now

- 2011-05-17 11:19:00: trucks seem so dangerous more so at noon to me now Trucks are tipped over

- 2011-05-17 11:20:00: That truck is irrepairable I always hated trucks especially at noon now

### Tentative summary on the trucks accident near a bridge on road 610

- Truck accident near a bridge on road 610 at noon. Multiple trucks involved in the accident? The trucks are tipped over, smashed, and burning.  They are spilling their cargo.

- We need to investigate this event in connection with the spread of the health problem.

- The content of the text is out of sync with the time. Maybe it's an hour off due to daylight saving time.

     (e.g., 2011-05-17 11:10:00: Trucks are smashed up 12pm and sitting on 610 fun fun)


# Analysis for 2011-05-16

In [276]:
def _():
    keywords = 'comic convention bomb threats baseball'.split(' ')
    posts = query_posts(posts=Posts.loc['2011-05-16'], keywords=keywords)
    label_posts_with_keywords(posts, 'keyword', keywords)
    display(create_map(query_posts(posts=posts), color='keyword'))

_()

# Two areas of interest

Multiple events went on this day.

1. A baseball game at C1 area.  Probably there is a baseball studium here.

1. A comic related gathering at g1 area. Probably there is a convention ceter here.

1. Many posts reporting "bomb" and "threats" in the western halve of the city center. Needs further investigation.

In [337]:
event_report('Bomb at the city center', '2011-05-16', 'f1:i3', 'bomb threats'.split(' '))

### Bomb at the city center (2042 messages)

0                                                      
1                burn won set wanna coming ll know just
2     bkuz checkpoint dozens wounds controlled ap dr...
                            ...                        
20    senor chino baja grub tacos shrimp fish bomb e...
21    bomb bus armed conspiracy costa rubicon quiz d...
22    bomb wanda lbvs droppin iraqi crackers eatin o...
Name: topic, Length: 23, dtype: object

In [323]:
BombPosts = query_posts(posts=Posts.loc['2011-05-16 18:45':'2011-05-16 19:00'], area='f1:i3', keywords='bomb'.split(' '))
summarize_posts('2011-05-16 News', BombPosts)
show_post_texts(BombPosts)

### 2011-05-16 News (48 messages)

- 2011-05-16 18:45:00: I suggest everyone be very cautious today the news has told me bombs are being deployed

- 2011-05-16 18:45:00: someone heard bombs may be in the city could they be just rumors?

- 2011-05-16 18:46:00: why do people have to do this someone has said a huge explosion may happen if there is really a bomb

- 2011-05-16 18:47:00: why do people have to do this I said bombs are being deployed

- 2011-05-16 18:48:00: everyone should be ready for the worst the news informed me there are bomb crews gearing up

- 2011-05-16 18:48:00: I recently heard bomb threats are rampant today taking cover is a good idea today

- 2011-05-16 18:49:00: everyone should be ready for the worst I informed me everyone should stay away from the places bomb threats have been reported at

- 2011-05-16 18:49:00: hopefully all of this has a nice peaceful end I has said bomb threats are rampant today

- 2011-05-16 18:50:00: someone has told me there is a bomb scare in the city I suggest everyone be very cautious today

- 2011-05-16 18:52:00: oh what a day... someone said there may be a bomb around

- 2011-05-16 18:53:00: someone recently heard there may be a bomb around danger everyone!

- 2011-05-16 18:54:00: I informed me bombs are being deployed this city needs to be on its toes

- 2011-05-16 18:55:00: I has said there is a bomb scare in the city oh what a day...

- 2011-05-16 18:57:00: I heard bombs may be in the city I wonder who could have started the threats

- 2011-05-16 18:59:00: can't believe what is going on someone heard a huge explosion may happen if there is really a bomb

## Bomb threads in the western halve of the city center

- This is a very weird event.  Many posts about bombs and threats in the western halve of the city center.  Until 2011-05-16 19:00 there were many posts on bomb threats and news reports on them.  After 2011-05-16 19:00, no mention of bomb threats. Some mentions about bomb crews and police but there seems no reporting how the situation was resolved.

- Probably the bomb threats are not related to the spread of the health problem.